<a href="https://colab.research.google.com/github/kdw9687/CapstonePBL_Predict_Movie/blob/main/ANN(4%EA%B0%80%EC%A7%80)_%2B_sigmoid_%2B_radam_%2B_%ED%95%99%EC%8A%B5%EB%A5%A0_%EA%B0%90%EC%86%8C___Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title 
!pip install -q keras-bert keras-rectified-adam
!wget -q https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!unzip -o uncased_L-12_H-768_A-12.zip
!pip install -U keras-tuner
!pip install -q -U tensorflow-addons

Archive:  uncased_L-12_H-768_A-12.zip
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-12_H-768_A-12/vocab.txt  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: uncased_L-12_H-768_A-12/bert_config.json  
Requirement already up-to-date: keras-tuner in /usr/local/lib/python3.7/dist-packages (1.0.2)


In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from google.colab import drive
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Input,Flatten,Activation
from tensorflow.keras.utils import to_categorical
drive.mount('/content/drive')
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
import sys
import numpy as np
from sklearn import preprocessing
from keras_radam import RAdam
np.set_printoptions(threshold=sys.maxsize)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def Mish(x):
  return x * K.tanh(K.softplus(x))

In [ ]:
file='/content/drive/MyDrive/Colab Notebooks/데이터 판다스용완.csv'

In [ ]:
data=pd.read_csv(file)

In [ ]:
data=data.dropna()

In [ ]:
data.drop(columns=['순번','장르','국적.1','연휴 및 기념일 상영 여부','수상'],inplace=True)
data.set_index('영화명',inplace=True)
target=data['전국 관객수']
target.dropna(inplace=True)
x=data.drop(columns=['전국 매출액','전국 관객수'])
y=target

In [ ]:
scaler=preprocessing.StandardScaler()
features_to_scale=['전국 스크린수','러닝타임','네티즌 평점']
scale=scaler.fit_transform(data[features_to_scale])
x[features_to_scale]=scale

In [ ]:
for i in range(len(y)):
  if y[i]>6000000:
    y[i]=3
  elif y[i]>3000000 and y[i]<=6000000:
    y[i]=2
  elif y[i]>1000000 and y[i]<=3000000:
    y[i]=1
  else:
    y[i]=0
  x=np.array(x)
  y=np.array(y)

In [ ]:
x_train,x_val,y_train,y_val=train_test_split(x,y,test_size=0.2,random_state=5)

In [ ]:
y_train=to_categorical(y_train)
y_val=to_categorical(y_val)

In [ ]:
inputs=Input(shape=(36,))
x=Flatten()(inputs)
x=Dense(800)(x)
x=Activation('sigmoid')(x)
x=Dense(400)(x)
x=Activation('sigmoid')(x)
x=Dense(4,activation='softmax')(x)
model=Model(inputs=inputs,outputs=x)

In [ ]:
model.compile(optimizer=RAdam(learning_rate=tf.compat.v1.train.exponential_decay(0.001,
0,5, 0.95,staircase=True)),loss='categorical_crossentropy',metrics=['acc'])

In [ ]:
history=model.fit(x_train,y_train,epochs=100,batch_size=40,validation_data=(x_val,y_val))

Epoch 1/100
20/20 [==============================] - 2s 12ms/step - loss: 1.5994 - acc: 0.2488 - val_loss: 1.1490 - val_acc: 0.5250
Epoch 2/100
20/20 [==============================] - 0s 5ms/step - loss: 1.1723 - acc: 0.4925 - val_loss: 1.0446 - val_acc: 0.5250
Epoch 3/100
20/20 [==============================] - 0s 5ms/step - loss: 1.1541 - acc: 0.4775 - val_loss: 1.0442 - val_acc: 0.5250
Epoch 4/100
16/20 [=======================>......] - ETA: 0s - loss: 1.1483 - acc: 0.4641

In [ ]:
import matplotlib.pyplot as plt
his_dict=history.history
loss=his_dict['loss']
val_loss=his_dict['val_loss']
epochs=range(1,len(loss)+1)
fig=plt.figure(figsize=(10,5))
ax1=fig.add_subplot(1,2,1)
ax1.plot(epochs,loss,color='blue',label='train_loss')
ax1.plot(epochs,val_loss,color='orange',label='val_loss')
ax1.set_title('train and val loss')
ax1.set_xlabel('epochs')
ax1.set_ylabel('loss')
ax1.legend()

acc=his_dict['acc']
val_acc=his_dict['val_acc']

ax2=fig.add_subplot(1,2,2)
ax2.plot(epochs,acc,color='blue',label='train_acc')
ax2.plot(epochs,val_acc,color='orange',label='val_acc')
ax2.set_title('train and val acc')
ax2.set_xlabel('epochs')
ax2.set_ylabel('acc')
ax2.legend()
plt.show()

In [ ]:
results=model.predict(x_val)
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

plt.figure(figsize=(7,7))
cm=confusion_matrix(np.argmax(y_val,axis=-1),np.argmax(results,axis=-1))
sns.heatmap(cm,annot=True,fmt='d',cmap='Blues')
plt.xlabel('predicted label')
plt.ylabel('true label')
plt.show()

In [ ]:
print('\n',classification_report(np.argmax(y_val,axis=-1),np.argmax(results,axis=-1)))

In [ ]:
model.summary()